In [1]:
import requests
import time
import random
import sqlite3
from bs4 import BeautifulSoup
import re
import random


user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
]


conn = sqlite3.connect('breachforum.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS forum_urls
                (id INTEGER PRIMARY KEY AUTOINCREMENT,
                url TEXT)''')


conn.commit()

url = "https://breachforums.vc/"

In [2]:
def scrape_forums(url):
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    all_links = soup.find_all("a")
    pattern = re.compile(r".*Forum-.*")
    # Filter and print the links that match the pattern
    for link in all_links:
        href = link.get("href")
        if href and pattern.match(href):
            print(href)
            cursor.execute('''INSERT INTO forum_urls (url) VALUES (?)''', (href,))
            conn.commit()

scrape_forums(url)

KeyboardInterrupt: 

In [3]:
def create_tables(c):
    cursor = c.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS topics
                (url TEXT,title text,user_id text,author text)''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS posts 
                (url TEXT,thread_title text,author text,post_text text,post_order integer,user_id text)''')
    c.commit()
create_tables(conn)

In [4]:
def scrape_topics(soup):
    topics = soup.find_all("tr", class_="inline_row")
    for topic in topics:
        try:
            elements = topic.find_all(class_="trow2")
            if elements is None:
              elements = topic.find_all(class_="trow1") 
            
            topic_url = elements[1].find("span", class_="subject_new").find("a").get("href")
            print("topic: ",topic_url)
            title = elements[1].find("span", class_="subject_new").find("a").text.strip()
            print("title: ",title)
            user_id = elements[1].find("span", class_="author").find("a").get("href").split("/")[-1]
            author = elements[1].find("span", class_="author").text.strip().split(" ")[-1].replace(",", "")
            print("author: ",author)
            cursor.execute("INSERT or ignore INTO topics VALUES (?,?,?,?)",(topic_url,title,user_id, author))
            conn.commit()
            print(f"Inserted {topic_url}")
        except Exception as e:
            print(e)
            continue



def scrape_forum_pages(forum_url):
    url = f"https://breachforums.vc/{forum_url[0]}"
    print(f"scraping {url}")
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    scrape_topics(soup)

#Because I have already scraped all pages I will only scrape the fist page, as there no need for dublicate threads
"""
    try:
        page_count = int(soup.find(class_="pages").text.strip().split("(")[-1].replace("):", ""))
    except Exception as e:
        page_count = None
        
    # Filter and print the links that match the pattern
    if page_count != None:
        
        for i in range(2,page_count):
            print(f"scraping page {i}")
            page = requests.get(url+f"?page={i}",headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            scrape_topics(soup)
             """

'\n    try:\n        page_count = int(soup.find(class_="pages").text.strip().split("(")[-1].replace("):", ""))\n    except Exception as e:\n        page_count = None\n        \n    # Filter and print the links that match the pattern\n    if page_count != None:\n        \n        for i in range(2,page_count):\n            print(f"scraping page {i}")\n            page = requests.get(url+f"?page={i}",headers=headers)\n            soup = BeautifulSoup(page.content, \'html.parser\')\n            scrape_topics(soup)\n             '

In [5]:
cursor.execute("select url from forum_urls ")
rows = cursor.fetchall()
for row in rows:
    scrape_forum_pages(row)
    time.sleep(5)

scraping https://breachforums.vc/Forum-Announcements
topic:  Thread-IMPORTANT-READ-Statement-regarding-Sebastien
title:  Statement regarding Sebastien
author:  ShinyHunters
Inserted Thread-IMPORTANT-READ-Statement-regarding-Sebastien
list index out of range
topic:  Thread-IMPORTANT-READ-Regarding-old-BreachForums-domains
title:  Regarding old BreachForums domains.
author:  Baphomet
Inserted Thread-IMPORTANT-READ-Regarding-old-BreachForums-domains
scraping https://breachforums.vc/Forum-Suggestions-Bug-Reports
list index out of range
topic:  Thread-Fix-shoutbox-autoscroll
title:  Fix shoutbox autoscroll
author:  kilob
Inserted Thread-Fix-shoutbox-autoscroll
list index out of range
topic:  Thread-Suggestion-reinstate-credit-owners-for-previously-supplied-breaches
title:  Suggestion: reinstate credit-owners for previously supplied breaches
author:  maraud3r
Inserted Thread-Suggestion-reinstate-credit-owners-for-previously-supplied-breaches
list index out of range
topic:  Thread-GOD-lounge
